## Setup

In [6]:
import numpy as np
import librosa
from sklearn.decomposition import FastICA
import soundfile as sf

## ICA

In [10]:
import numpy as np
import librosa
from sklearn.decomposition import FastICA
import soundfile as sf
import matplotlib.pyplot as plt

def extract_vocals_enhanced(input_mp3, output_vocals='vocals.wav', sr=44100):
    # 1. Load with higher quality settings
    try:
        x, sr = librosa.load(input_mp3, sr=sr, mono=False)
        if x.ndim == 1:
            x = np.vstack([x, x])  # Create stereo if mono
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

    # 2. Create enhanced channel set
    mid = 0.5 * (x[0] + x[1])  # Mid channel (vocals usually centered)
    side = 0.5 * (x[0] - x[1])  # Side channel
    X = np.vstack([x[0], x[1], mid, side])  # 4-channel input

    # 3. Perform ICA with more components
    ica = FastICA(n_components=4, max_iter=1000, random_state=42)
    sources = ica.fit_transform(X.T).T

    # 4. Better vocal identification
    def is_vocal_component(y):
        y_harmonic = librosa.effects.harmonic(y)
        features = {
            'harmonic': np.mean(librosa.feature.spectral_flatness(y=y_harmonic)),
            'percussive': 1 - np.mean(librosa.feature.spectral_flatness(y=librosa.effects.percussive(y))),
            'centroid': np.mean(librosa.feature.spectral_centroid(y=y))
        }
        return 0.6*features['harmonic'] + 0.3*features['percussive'] + 0.1*(1/features['centroid'])

    scores = [is_vocal_component(src) for src in sources]
    vocal_idx = np.argmax(scores)
    vocals = sources[vocal_idx]

    # 5. Advanced post-processing
    vocals = librosa.effects.harmonic(vocals, margin=8)  # Stronger harmonic extraction
    vocals = librosa.util.normalize(vocals) * 0.9  # Prevent clipping

    # 6. Save output
    sf.write(output_vocals, vocals, sr)
    print(f"Vocal extraction complete. Saved to {output_vocals}")
    
    return vocals

# Usage
extract_vocals_enhanced('Nti T9lchi.mp3', 'extracted_vocals.wav')

KeyboardInterrupt: 